# RAP Example Python Pipeline - Interactive Exercise

ADD THE GOOGLE COLAB LINK HERE

## Intro

This notebook will show you how straight-forward it is to do an analytical pipeline in Python.

The core of any of piece of analytical work is to:
- load some data
- do something to do that, e.g. process it, do some analysis
- create some output

This notebook will go briefly through each of these showing *one* way of doing it in Python (there are many more!). 

Open this notebook in google colab and have a play - try changing bits and see what happens!

**NOTE**: to make the workshop more straight forward, we haven't completely followed good practice. If you want to see a pipeline how it should be, well laid out and modularised, [see our Example Python pipeline](https://github.com/NHSDigital/RAP_example_pipeline_python).

## Setup

We will need to install a few things before we can get going.

First, if this is running in Google Colab, we need to clone the repo and install the right python packages.

In [ ]:
# this forces google collab to install the dependencies
if "google.colab" in str(get_ipython()):
    print("Running on Colab")
    !git clone https://github.com/NHSDigital/RAP_example_pipeline_python.git -q
    %cd RAP_example_pipeline_python
    !pip install -r requirements.txt -q -q

Next we need to import the right libraries for this piece of work:

In [ ]:
# These libraries will help us download the file
import zipfile
import io
from pathlib import Path
import requests


import pandas as pd # this allows us to work with dataframes
import matplotlib.pyplot as plt # this allows us to create some graphs


## Config

Quite often, when we create an analytical process in code, there will be parts of the code that we need to update - dates, URLs, file paths, and so on.
 
We usually put these in a separate file called a config file. That way, all our settings are in one place. Here we'll just put our config settings in this cell below.

In [ ]:
zip_file_url = "https://files.digital.nhs.uk/assets/Services/Artificial%20data/Artificial%20HES%20final/artificial_hes_ae_202302_v1_sample.zip"
path_to_downloaded_data = "data_in/artificial_hes_ae_202302_v1_sample.zip/artificial_hes_ae_202302_v1_sample/artificial_hes_ae_2122.csv"

# The column(s) we are going to investigate
col_to_group = "AEARRIVALMODE"


## Load Data

First we will load the data: we're going to use an artificial fake version of the NHS Hospital Episode Statistics Accident and Emergency (HES AE) data from 2003. 


This code:
- gets the location of the data from the config file
- downloads the CSV
- saves it to our data_in folder

This is just an example - in another setting we could make it load the data from a SQL server, or from a database, S3 bucket, etc.

In [ ]:
filename = Path(zip_file_url).name
output_path = f"data_in/{filename}"

response = requests.get(zip_file_url, stream=True,timeout=3600)
downloaded_zip = zipfile.ZipFile(io.BytesIO(response.content))
downloaded_zip.extractall(output_path)


Now we need to read in our data and store it so we can continue to use and manipulate it. We'll use the pandas method read_csv to turn the data within the csv file to a pandas dataframe (commonly referred to as df).

A pandas DataFrame is a two-dimensional, labeled data structure in Python, similar to a table in a database or an Excel spreadsheet, that allows for the storage and manipulation of data across rows and columns

In [ ]:
df_hes = pd.read_csv(path_to_downloaded_data)

Let's see what this data looks like, and pull the first 5 rows:

In [ ]:
df_hes.head(5)

## Processing

Now the fun part - we get to do some interesting processing on the data.

Let's group the dataframe by the specified columns in cols_to_group, counts the number of rows in each group, and create a new column "Count" to show this.

In [ ]:
df_hes_grouped = (df_hes
    .groupby(col_to_group) # we state the column we want to group by. We defined col_to_group above!
    .size() # Here we tell python we want to count the number of each value in the column. It's like count() in SQL
    .to_frame('Count') # This turns the counts into a dataframe, and gives a title to out counts column
)

display(df_hes_grouped)

It's a little confusing what those numbers mean, so let's put them into plain English

In [ ]:
ae_arrival_mode_map = {
    1: "Ambulance",
    2: "Other",
    9: "Not known"
}
df_hes_grouped = df_hes_grouped.rename(index=ae_arrival_mode_map)
print(df_hes_grouped)

That's better!

## Visualising

Now it's time to take our dataframe and turn it into something nice to look at and easy to interpret!

In [ ]:
df_hes_grouped.plot.bar(rot = 0)
plt.title(f"Count by {col_to_group}") # Here we add a title. Try changing it! You can put anything you want, it just has to be in quotes
plt.show()

## Changing Outputs

Due to how the code is written, you can easily swap what column you are investigating. Under "Config", try swapping the column in col_to_group with one of the other columns listed here. Remember to update the name mapping too.

AEATTENDDISP - Attendance Disposal

AEINCLOCTYPE - Where incident occurred

AEREFSOURCE - Source of referral

ETHNOS - Ethnicity of patient

RESGOR_ONS - Region of Residence

SEX - Patient's Sex

Check out the HES Technical Output Specification to understand what each column mean and the possible options.

https://digital.nhs.uk/data-and-information/data-tools-and-services/data-services/hospital-episode-statistics/hospital-episode-statistics-data-dictionary

Then just run the notebook again to see your results!




## Advanced

You could try grouping by two or more columns, to understand the data a little deeper.

How about we make a grouped bar chart?

### Config

This time we'll need to list two columns. Pick two columns from the list above and put them into the square brackets below:

In [ ]:
cols_to_group = []

### Grouped bar chart

Copy the code below into the cell underneath. Most of the code is ready for you, but there are some bits missing.

Can you fill in the ????s to complete the grouped bar chart?

For bonus points, try adding a title to the chart!


```
df_hes_grouped_bar = df_hes[????]

df_hes_grouped_bar = df_hes_grouped_bar.groupby(????).size().unstack()

df_hes_grouped_bar.plot.bar(rot = 0)

plt.show()
```